<a href="https://colab.research.google.com/github/philsML/Classification_by_Keyword/blob/master/MLwP_Group_Projects_Keywords_Application_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
import matplotlib.pyplot as plt
from google.colab import drive
import random
from pprint import pprint
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV, GridSearchCV, cross_val_score, ShuffleSplit, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample

In [0]:
#Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression, BayesianRidge, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB

In [0]:
random.seed(123)

In [4]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#switched to v11 to include Jakob's changes - please re-run the preprocessing code, if colab can't find the file
data=pd.read_csv('/content/drive/My Drive/ML Python_Group Project/Table for Analysis_v11.csv',sep=";")
data.rename({"Unnamed: 0":"ID"}, axis='columns',inplace=True)
cols = data.columns.to_list()

In [6]:
rf = RandomForestClassifier(n_estimators=100,min_samples_split=250,min_samples_leaf=5,max_features="auto",max_depth=40,bootstrap=True)
rf.fit(data[data.columns[4:]],np.array(data.iloc[:,3:4]).flatten())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=250,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
RFR = RandomForestRegressor(n_estimators = 200,
                            max_depth = 50, 
                            min_samples_split = 15,
                            min_samples_leaf = 10)
RFR.fit(data[data.columns[4:]],np.array(data.iloc[:,1:2]).flatten())

In [0]:
RFC_2 = RandomForestClassifier(n_estimators = 50,
                            max_depth = 50, 
                            min_samples_split = 15,
                            min_samples_leaf = 10)
RFC_2.fit(data[data.columns[4:]],np.array(data.iloc[:,2:3]).flatten())

In [0]:
def to_age_cat(input_variable):
  if input_variable=="A":
    return("14-24")
  elif input_variable=="B":
    return("25-34")
  elif input_variable=="C":
    return("35-44")
  elif input_variable=="D":
    return("45-54")
  elif input_variable=="E":
    return("55-64")    
  else:
    return("65-99")

In [0]:
def to_sex(input_variable):
  if input_variable==1:
    return("M")
  else:
    return("F")

In [0]:
input_files = ['/content/drive/My Drive/ML Python_Group Project/apply_500k.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_1-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_1-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_2-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_2-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/apply_3-0M.csv']
output_files = ['/content/drive/My Drive/ML Python_Group Project/scored_500k.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_1-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_1-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_2-0M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_2-5M.csv',
               '/content/drive/My Drive/ML Python_Group Project/scored_3-0M.csv']

In [0]:
counter=0
for inputs,outputs in zip(input_files,output_files):
  
  apply = pd.read_csv(inputs,sep=";")
  apply.drop("Unnamed: 0.1",axis="columns",inplace=True)
  apply.rename({"Unnamed: 0":"ID"}, axis='columns',inplace=True)

  pred_sex = rf.predict(apply[apply.columns[1:]])
  pred_age_reg = RFR.predict(apply[apply.columns[1:]])
  pred_age_reg_bins = pd.cut(pred_age_reg, bins=[0, 24, 34, 44, 54, 64, 99])
  pred_age_cat_bins = RFC_2.predict(apply[apply.columns[1:]])

  prediction_result = pd.DataFrame(pred_age_cat_bins,columns=["Predicted_Age_Clusters_Cat"],index=apply["ID"])
  prediction_result["Predicted_Age_Clusters_Reg"]=pred_age_reg_bins
  prediction_result["Predicted_Sex"]=pred_sex

  prediction_result.Predicted_Age_Clusters_Cat.apply(lambda x: to_age_cat(x))
  prediction_result.Predicted_Sex = prediction_result.Predicted_Sex.apply(lambda x: to_sex(x))

  prediction_result.to_csv(outputs,sep=";")
  
  del(apply,prediction_result,pred_sex,pred_age_reg,pred_age_reg_bins,pred_age_cat_bins)

  counter=counter+1
  print(counter,"/6 done")